In [1]:
import pandas as pd

# Read the Excel file
df = pd.read_excel('Dades/IQ_Cancer_Endometrio_merged_NMSP.xlsx')

# Display the first few rows
print(f"Número de columnas: {df.shape[1]}")
print(f"Número de filas: {df.shape[0]}")
df.head()

Número de columnas: 189
Número de filas: 163


,codigo_participante,recidiva,recidiva_exitus,diferencia_dias_reci_exit,causa_muerte,f_diag,fecha_de_recidi,f_muerte,visita_control,Ultima_fecha,...,bt_realPac,ini_bqt_rt,final_bqt_rt,qt,Tratamiento_sistemico_realizad,inicio_qmt,fecha_final_qmt,otros_tt,tt_o_f_ini,tt_o_f_fin
0,4,0,0,626.0,NaN,2023-11-03,NaN,NaN,21/07/2025,2025-07-21,...,NaN,NaN,NaN,1.0,2.0,14/02/2024,04/04/2024,NaN,NaN,NaN
1,7,0,0,244.0,1.0,2019-10-24,NaN,08/04/2021,24/06/2020,2020-06-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,0,0,1769.0,NaN,2020-07-16,NaN,NaN,20/05/2025,2025-05-20,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,9,1,1,1003.0,NaN,2021-02-22,22/11/2023,NaN,27/10/2025,2023-11-22,...,1.0,08/10/2021,12/11/2021,1.0,2.0,06/05/2021,20/08/2021,NaN,NaN,NaN
4,10,0,0,1917.0,NaN,2019-11-15,NaN,NaN,13/02/2025,2025-02-13,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Remove columns that have 100% missing values (only NaN)
df_cleaned = df.dropna(axis=1, how='all')

print(f"Número de columnas antes: {df.shape[1]}")
print(f"Número de columnas después: {df_cleaned.shape[1]}")
print(f"Columnas eliminadas: {df.shape[1] - df_cleaned.shape[1]}")

# Update the main dataframe
# Update the main dataframe
df = df_cleaned

# Show which columns were removed
columns_removed = set(df_cleaned.columns) ^ set(pd.read_excel('Dades/IQ_Cancer_Endometrio_merged_NMSP.xlsx').columns)
if columns_removed:
	print(f"\nColumnas eliminadas ({len(columns_removed)}):")
	for col in sorted(columns_removed):
		print(f"  - {col}")
else:
	print("\nNo se eliminaron columnas (ninguna tenía 100% valores faltantes)")

Número de columnas antes: 189
Número de columnas después: 183
Columnas eliminadas: 6

Columnas eliminadas (6):
  - otro_ttIQ_recid
  - tabla_de_estadi
  - tabla_de_riesgo
  - tiempo_transcur
  - tt_o_f_fin
  - tt_o_f_ini


In [3]:
# Columnas de fechas / línea temporal
fechas_cols = [
    "FN",
    "f_1v",
    "f_diag",
    "f_tto_NA",
    "fecha_qx",
    "inicio_qmt",
    "fecha_final_qmt",
    "ini_bqt_rt",
    "final_bqt_rt",
    "fecha_de_recidi",
    "f_muerte",
    "visita_control",
    "Ultima_fecha",
]

# Filtra solo estas columnas
fechas_cols = [c for c in fechas_cols if c in df.columns]

df_fechas = df[fechas_cols].copy()

# Convert all date columns to datetime
for col in fechas_cols:
	df_fechas[col] = pd.to_datetime(df_fechas[col], errors='coerce', dayfirst=True)

# Update the main dataframe with converted datetime columns
df[fechas_cols] = df_fechas

df[fechas_cols].head()

,FN,f_1v,f_diag,f_tto_NA,fecha_qx,inicio_qmt,fecha_final_qmt,ini_bqt_rt,final_bqt_rt,fecha_de_recidi,f_muerte,visita_control,Ultima_fecha
0,1955-10-12,2023-11-03,2023-11-03,NaT,2023-11-16,2024-02-14,2024-04-04,NaT,NaT,NaT,NaT,2025-07-21,2025-07-21
1,1926-09-24,2019-04-11,2019-10-24,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2021-04-08,2020-06-24,2020-06-24
2,1947-11-26,2020-07-15,2020-07-16,NaT,2020-09-14,NaT,NaT,NaT,NaT,NaT,NaT,2025-05-20,2025-05-20
3,1942-10-15,2021-02-11,2021-02-22,NaT,2021-03-18,2021-05-06,2021-08-20,2021-10-08,2021-11-12,2023-11-22,NaT,2025-10-27,2023-11-22
4,1951-07-06,2019-11-12,2019-11-15,NaT,2019-12-13,NaT,NaT,NaT,NaT,NaT,NaT,2025-02-13,2025-02-13


In [4]:
# Special case: Fix incorrect dates for participants with missing FN
# The date had incorrect month values (60 instead of 6, 70 instead of 7)

# Get the codigo_participante where FN is missing
missing_fn_codes = df[df['FN'].isna()]['codigo_participante'].tolist()

print(f"Códigos de participantes con FN faltante: {missing_fn_codes}")

if missing_fn_codes:
	for code in missing_fn_codes:
		df.loc[df['codigo_participante'] == code, 'FN'] = pd.to_datetime('1961-06-05')
		df.loc[df['codigo_participante'] == code, 'f_1v'] = pd.to_datetime('2024-07-10')
		
		print(f"\nFechas corregidas para codigo_participante {code}:")
		print(f"Nueva FN: {df.loc[df['codigo_participante'] == code, 'FN'].values[0]}")
		print(f"Nueva f_1v: {df.loc[df['codigo_participante'] == code, 'f_1v'].values[0]}")

Códigos de participantes con FN faltante: [265]

Fechas corregidas para codigo_participante 265:
Nueva FN: 1961-06-05T00:00:00.000000000
Nueva f_1v: 2024-07-10T00:00:00.000000000


In [5]:
# Keep only the specified date columns
fechas_a_mantener = ["FN", "fecha_qx", "fecha_de_recidi", "f_muerte", "visita_control", "Ultima_fecha"]

# Identify date columns to drop
fechas_a_eliminar = [col for col in fechas_cols if col not in fechas_a_mantener and col in df.columns]

# Drop the unwanted date columns
df = df.drop(columns=fechas_a_eliminar)

print(f"Columnas de fechas eliminadas: {len(fechas_a_eliminar)}")
print(f"Columnas eliminadas:")
for col in fechas_a_eliminar:
	print(f"  - {col}")

print(f"\nColumnas de fechas mantenidas:")
for col in fechas_a_mantener:
	if col in df.columns:
		print(f"  - {col}")

print(f"\nNúmero de columnas después: {df.shape[1]}")

Columnas de fechas eliminadas: 7
Columnas eliminadas:
  - f_1v
  - f_diag
  - f_tto_NA
  - inicio_qmt
  - fecha_final_qmt
  - ini_bqt_rt
  - final_bqt_rt

Columnas de fechas mantenidas:
  - FN
  - fecha_qx
  - fecha_de_recidi
  - f_muerte
  - visita_control
  - Ultima_fecha

Número de columnas después: 176


In [6]:
# Identify text/comment columns by name
text_comment_columns = [
	'ap_comentarios',
	'centro_tratPrima',
	'comentarios',
	'histo_otros',
	'otra_histo',
	'otras',
	'otras_especifi',
	'otros_tt',
	'recid_super_1',
	'reintervencion_motivo',
	'tec_Qx_avanz'
]

# Filter only columns that exist in the dataframe
text_comment_columns = [col for col in text_comment_columns if col in df.columns]

print(f"Columnas de texto/comentarios identificadas: {len(text_comment_columns)}")
print("\nColumnas:")
for col in sorted(text_comment_columns):
	print(f"  - {col}")

# Drop the text/comment columns from the main dataframe
df = df.drop(columns=text_comment_columns)

print(f"\nNúmero de columnas después de eliminar texto/comentarios: {df.shape[1]}")
df.head()


Columnas de texto/comentarios identificadas: 11

Columnas:
  - ap_comentarios
  - centro_tratPrima
  - comentarios
  - histo_otros
  - otra_histo
  - otras
  - otras_especifi
  - otros_tt
  - recid_super_1
  - reintervencion_motivo
  - tec_Qx_avanz

Número de columnas después de eliminar texto/comentarios: 165


,codigo_participante,recidiva,recidiva_exitus,diferencia_dias_reci_exit,causa_muerte,fecha_de_recidi,f_muerte,visita_control,Ultima_fecha,FN,...,grupo_de_riesgo_definitivo,Tributaria_a_Radioterapia,rdt,rt_dosis,n_doisis_rt,moti_no_RT,bqt,bt_realPac,qt,Tratamiento_sistemico_realizad
0,4,0,0,626.0,NaN,NaT,NaT,2025-07-21,2025-07-21,1955-10-12,...,5.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,1.0,2.0
1,7,0,0,244.0,1.0,NaT,2021-04-08,2020-06-24,2020-06-24,1926-09-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,0,0,1769.0,NaN,NaT,NaT,2025-05-20,2025-05-20,1947-11-26,...,1.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN
3,9,1,1,1003.0,NaN,2023-11-22,NaT,2025-10-27,2023-11-22,1942-10-15,...,5.0,1.0,2.0,2.0,25.0,NaN,1.0,1.0,1.0,2.0
4,10,0,0,1917.0,NaN,NaT,NaT,2025-02-13,2025-02-13,1951-07-06,...,1.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN


In [7]:
# Get the codigo_participante where edad is 0
edad_0_codes = df[df['edad'] == 0]['codigo_participante'].tolist()

print(f"Códigos de participantes con edad = 0: {edad_0_codes}")
print(f"Número de participantes con edad = 0: {len(edad_0_codes)}")

# Show more details about these participants
if edad_0_codes:
	print("\nDetalles de los participantes con edad = 0:")
	print(df[df['edad'] == 0][['codigo_participante', 'FN', 'fecha_qx', 'edad']])

# Drop rows where codigo_participante is in edad_0_codes
df = df[~df['codigo_participante'].isin(edad_0_codes)]
print(f"\nNúmero de filas después de eliminar casos con edad = 0: {df.shape[0]}")
print(f"Filas eliminadas: {len(edad_0_codes)}")

Códigos de participantes con edad = 0: [81, 153]
Número de participantes con edad = 0: 2

Detalles de los participantes con edad = 0:
     codigo_participante         FN   fecha_qx  edad
48                    81 2022-03-09 2022-02-23   0.0
101                  153 2018-03-11 2018-05-04   0.0

Número de filas después de eliminar casos con edad = 0: 161
Filas eliminadas: 2


In [8]:
# Get the codigo_participante where fecha_qx is NaN
fecha_qx_na_codes = df[df['fecha_qx'].isna()]['codigo_participante'].tolist()

print(f"Códigos de participantes con fecha_qx faltante: {fecha_qx_na_codes}")
print(f"Número de participantes con fecha_qx faltante: {len(fecha_qx_na_codes)}")

# Show more details about these participants
if fecha_qx_na_codes:
	print("\nDetalles de los participantes con fecha_qx faltante:")
	print(df[df['fecha_qx'].isna()][['codigo_participante', 'FN', 'fecha_qx', 'edad', 'tto_1_quirugico']])
 
# Drop rows where codigo_participante is in fecha_qx_na_codes
df = df[~df['codigo_participante'].isin(fecha_qx_na_codes)]
print(f"\nNúmero de filas después de eliminar casos con fecha_qx faltante: {df.shape[0]}")
print(f"Filas eliminadas: {len(fecha_qx_na_codes)}")

Códigos de participantes con fecha_qx faltante: [7, 18, 30, 54, 95, 100, 132, 135, 164, 200, 216, 222, 223]
Número de participantes con fecha_qx faltante: 13

Detalles de los participantes con fecha_qx faltante:
     codigo_participante         FN fecha_qx  edad  tto_1_quirugico
1                      7 1926-09-24      NaT  92.0              0.0
7                     18 1981-10-03      NaT  41.0              0.0
16                    30 1941-01-27      NaT  78.0              0.0
32                    54 1956-02-21      NaT  67.0              1.0
58                    95 1930-07-05      NaT  89.0              0.0
62                   100 1965-12-08      NaT  54.0              1.0
88                   132 1981-10-21      NaT  39.0              0.0
91                   135 1929-04-13      NaT  89.0              0.0
105                  164 1933-08-15      NaT  87.0              0.0
122                  200 1967-11-11      NaT  55.0              0.0
130                  216 1937-08-23     

In [9]:
# Calculate age at surgery
df['edad_en_cirugia'] = (df['fecha_qx'] - df['FN']).dt.days / 365.25

print("\nEdad en cirugía:")
print(df['edad_en_cirugia'].describe())

# Print negative ages in surgery
negative_ages_qx = df[df['edad_en_cirugia'] < 0]
if len(negative_ages_qx) > 0:
	print("\nEdades en cirugía negativas:")
	print(negative_ages_qx[['codigo_participante', 'FN', 'f_diag', 'fecha_qx', 'edad_en_diagnostico', 'edad_en_cirugia']])
else:
	print("\nNo hay edades en cirugía negativas")


Edad en cirugía:
count    148.000000
mean      63.051705
std       11.823506
min       31.039014
25%       55.132786
50%       63.822040
75%       71.632444
max       88.780287
Name: edad_en_cirugia, dtype: float64

No hay edades en cirugía negativas


In [10]:
n_before = df.shape[0]

# Calculate event_date as the minimum of fecha_de_recidi and f_muerte (ignoring NaT)
rec_ok = df['fecha_de_recidi'].where(df['fecha_de_recidi'] >= df['fecha_qx'])
mur_ok = df['f_muerte'].where(df['f_muerte'] >= df['fecha_qx'])
df['event_date'] = pd.concat([rec_ok, mur_ok], axis=1).min(axis=1)

# Create event indicator: 1 if event_date exists, 0 otherwise
df['event'] = df['event_date'].notna().astype(int)

# Calculate end_date: event_date if event=1, otherwise visita_control
df['end_date'] = df['event_date'].where(df['event'] == 1, df['visita_control'])
df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')

# Calculate time_days: end_date - fecha_qx
df['time_days'] = (df['end_date'] - df['fecha_qx']).dt.days

print(f"Número de filas antes de filtrar: {df.shape[0]}")
print(f"\nDistribución de event:")
print(df['event'].value_counts())
print(f"\nEstadísticas de time_days:")
print(df['time_days'].describe())

# Check for missing or negative time_days
missing_time = df['time_days'].isna().sum()
negative_time = (df['time_days'] < 0).sum()

print(f"\nCasos con time_days faltante: {missing_time}")
print(f"Casos con time_days negativo: {negative_time}")

bad_censor = (df['event'] == 0) & df['visita_control'].notna() & (df['visita_control'] < df['fecha_qx'])
print(f"Casos con censura incoherente (visita_control < fecha_qx y event=0): {int(bad_censor.sum())}")

if missing_time > 0 or negative_time > 0:
	print("\nDetalles de casos con time_days faltante o negativo:")
	problematic_cases = df[(df['time_days'].isna()) | (df['time_days'] < 0)]
	print(problematic_cases[['codigo_participante', 'fecha_qx', 'fecha_de_recidi', 'f_muerte', 'visita_control', 'event', 'event_date', 'end_date', 'time_days']])

if int(bad_censor.sum()) > 0:
    print("\nDetalles de casos con censura incoherente:")
    print(df.loc[bad_censor, ['codigo_participante', 'fecha_qx', 'visita_control', 'event', 'end_date', 'time_days']])

# Filter out cases with missing or negative time_days
df = df[(df['time_days'].notna()) & (df['time_days'] >= 0)]
n_after = df.shape[0]

print(f"\nNúmero de filas después de filtrar: {df.shape[0]}")
print("Filas eliminadas:", n_before - n_after)

Número de filas antes de filtrar: 148

Distribución de event:
event
0    111
1     37
Name: count, dtype: int64

Estadísticas de time_days:
count     148.000000
mean     1053.972973
std       667.537417
min      -338.000000
25%       462.250000
50%      1004.500000
75%      1685.750000
max      2481.000000
Name: time_days, dtype: float64

Casos con time_days faltante: 0
Casos con time_days negativo: 1
Casos con censura incoherente (visita_control < fecha_qx y event=0): 1

Detalles de casos con time_days faltante o negativo:


     codigo_participante   fecha_qx fecha_de_recidi f_muerte visita_control  \
146                  244 2025-04-19             NaT      NaT     2024-05-16   

     event event_date   end_date  time_days  
146      0        NaT 2024-05-16       -338  

Detalles de casos con censura incoherente:
     codigo_participante   fecha_qx visita_control  event   end_date  \
146                  244 2025-04-19     2024-05-16      0 2024-05-16   

     time_days  
146       -338  

Número de filas después de filtrar: 147
Filas eliminadas: 1


In [11]:
# Validate end_date against Ultima_fecha
# Check how many cases have mismatches between end_date and Ultima_fecha

# Filter cases where Ultima_fecha is not null
df_with_ultima = df[df['Ultima_fecha'].notna()].copy()

print(f"Casos con Ultima_fecha no nula: {len(df_with_ultima)}")

# Compare end_date with Ultima_fecha
df_with_ultima['fecha_match'] = (df_with_ultima['end_date'] == df_with_ultima['Ultima_fecha'])

mismatches = df_with_ultima[~df_with_ultima['fecha_match']]

print(f"\nCasos donde end_date != Ultima_fecha: {len(mismatches)}")
print(f"Porcentaje de coincidencia: {(df_with_ultima['fecha_match'].sum() / len(df_with_ultima)) * 100:.2f}%")

if len(mismatches) > 0:
	print("\nDetalles de casos con discrepancias:")
	print(mismatches[['codigo_participante', 'fecha_qx', 'fecha_de_recidi', 'f_muerte', 'visita_control', 'Ultima_fecha', 'event', 'event_date', 'end_date', 'time_days']].head(10))
	
	# Calculate time difference in days
	mismatches['diferencia_dias'] = (mismatches['end_date'] - mismatches['Ultima_fecha']).dt.days
	print(f"\nEstadísticas de diferencia en días:")
	print(mismatches['diferencia_dias'].describe())
else:
	print("\nNo hay discrepancias entre end_date y Ultima_fecha")

# Note: We use end_date as the "truth" for survival analysis, 
# but this validation helps detect potential data quality issues

Casos con Ultima_fecha no nula: 147

Casos donde end_date != Ultima_fecha: 5
Porcentaje de coincidencia: 96.60%

Detalles de casos con discrepancias:
    codigo_participante   fecha_qx fecha_de_recidi   f_muerte visita_control  \
6                    12 2019-07-04             NaT 2024-06-24     2023-12-13   
36                   62 2023-06-22             NaT 2024-06-11     2024-05-22   
61                   98 2020-11-30             NaT 2022-01-22     2022-01-13   
72                  114 2019-12-02             NaT 2021-07-06     2021-02-04   
98                  148 2018-08-29             NaT 2020-02-26     2020-01-26   

   Ultima_fecha  event event_date   end_date  time_days  
6    2023-12-13      1 2024-06-24 2024-06-24       1817  
36   2024-05-22      1 2024-06-11 2024-06-11        355  
61   2022-01-13      1 2022-01-22 2022-01-22        418  
72   2021-02-04      1 2021-07-06 2021-07-06        582  
98   2020-01-26      1 2020-02-26 2020-02-26        546  

Estadísticas de dife

/tmp/ipykernel_15000/2736080700.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mismatches['diferencia_dias'] = (mismatches['end_date'] - mismatches['Ultima_fecha']).dt.days


In [12]:
# Drop the date columns that were kept for validation purposes
# Now that validation is complete, we only need the derived variables (event_date, end_date, time_days)

fechas_drop = fechas_a_mantener.copy()
fechas_drop = fechas_drop + ['codigo_participante', 'event_date', 'end_date']

df = df.drop(columns=fechas_drop)

print(f"Columnas de fechas eliminadas: {len(fechas_drop)}")
print(f"Columnas eliminadas:")
for col in fechas_drop:
	print(f"  - {col}")

print(f"\nNúmero de columnas después: {df.shape[1]}")
print(f"Número de filas: {df.shape[0]}")

# Verify the survival analysis variables are still present
survival_vars = ['event', 'edad_en_cirugia', 'time_days']
print(f"\nVariables de análisis de supervivencia presentes:")
for var in survival_vars:
	if var in df.columns:
		print(f"  ✓ {var}")
	else:
		print(f"  ✗ {var} (FALTA)")


Columnas de fechas eliminadas: 9
Columnas eliminadas:
  - FN
  - fecha_qx
  - fecha_de_recidi
  - f_muerte
  - visita_control
  - Ultima_fecha
  - codigo_participante
  - event_date
  - end_date

Número de columnas después: 161
Número de filas: 147

Variables de análisis de supervivencia presentes:
  ✓ event
  ✓ edad_en_cirugia
  ✓ time_days


In [13]:
# Define numeric columns explicitly
numeric = [
    'diferencia_dias_reci_exit',
	'edad',
	'imc',
	'valor_de_ca125',
	'ap_gPaor_total',
	'ap_gPaor_afect_total',
	'ciclos_tto_NAdj',
	'dias_de_ingreso',
	'n_doisis_rt',
	'n_gangP_afec',
	'n_ganPaor_InfrM_afec',
	'n_ganPaor_Sup_afec',
	'n_GC_Afect',
	'n_resec_Intes',
	'n_total_ganCent',
	'n_total_ganPaor_infra',
	'n_total_ganPaor_supr',
	'n_total_GC',
	'perdida_hem_cc',
	'recep_est_porcent',
	'rece_de_Ppor',
	'tamano_tumoral',
	'tiempo_qx',
	'transf_GRC',
	'numero_de_recid'
]

# Filter only columns that exist in the dataframe
numeric = [col for col in numeric if col in df.columns]

print(f"Columnas numéricas continuas: {len(numeric)}")
print("\nColumnas:")
for col in sorted(numeric):
	print(f"  - {col}")

# Show statistics for these columns
if numeric:
	print("\nEstadísticas de las columnas numéricas:")
	# Convert to numeric to ensure proper type
	for col in numeric:
		df[col] = pd.to_numeric(df[col], errors='coerce')
	
	print(df[numeric].describe())

Columnas numéricas continuas: 24

Columnas:
  - ap_gPaor_total
  - ciclos_tto_NAdj
  - dias_de_ingreso
  - diferencia_dias_reci_exit
  - edad
  - imc
  - n_GC_Afect
  - n_doisis_rt
  - n_ganPaor_InfrM_afec
  - n_ganPaor_Sup_afec
  - n_gangP_afec
  - n_resec_Intes
  - n_total_GC
  - n_total_ganCent
  - n_total_ganPaor_infra
  - n_total_ganPaor_supr
  - numero_de_recid
  - perdida_hem_cc
  - rece_de_Ppor
  - recep_est_porcent
  - tamano_tumoral
  - tiempo_qx
  - transf_GRC
  - valor_de_ca125

Estadísticas de las columnas numéricas:


       diferencia_dias_reci_exit        edad         imc  valor_de_ca125  \
count                 145.000000  147.000000  145.000000       17.000000   
mean                 1134.841379   62.598639   31.002276       74.617647   
std                   654.301214   11.898924    7.807467      104.521955   
min                    65.000000   30.000000   16.700000        3.700000   
25%                   537.000000   55.000000   25.000000       14.200000   
50%                  1082.000000   63.000000   30.200000       24.200000   
75%                  1708.000000   71.500000   35.500000       83.900000   
max                  2671.000000   88.000000   56.100000      370.800000   

       ap_gPaor_total  ciclos_tto_NAdj  dias_de_ingreso  n_doisis_rt  \
count      114.000000         3.000000       136.000000    77.000000   
mean         4.377193         5.000000         2.992647     8.792208   
std          7.589455         1.732051         3.280687    12.350514   
min          0.000000      

In [14]:
# Define categorical columns as all columns except the numeric ones and survival variables
categorical = [col for col in df.columns if col not in numeric + ['event', 'edad_en_cirugia', 'time_days', 'codigo_participante']]

print(f"Columnas categóricas: {len(categorical)}")
print("\nColumnas:")
for col in sorted(categorical):
	print(f"  - {col}")

# Convert categorical columns to categorical dtype
for col in categorical:
	df[col] = df[col].astype('category')

print("\nTipos de datos después de la conversión:")
print(f"Columnas categóricas: {len([col for col in df.columns if df[col].dtype.name == 'category'])}")
print(f"Columnas numéricas: {len([col for col in df.columns if df[col].dtype in ['float64', 'int64']])}")

Columnas categóricas: 134

Columnas:
  - AP_centinela_pelvico
  - AP_ganPelv
  - AP_glanPaor
  - Anexectomia
  - FIGO2023
  - Grado
  - Local_Gan_Paor
  - Motivo_de_conversion_a_LPT_r01
  - Motivo_de_conversion_a_LPT_r02
  - Motivo_de_conversion_a_LPT_r03
  - Motivo_de_conversion_a_LPT_r04
  - Movilizador_uterino
  - Perforacion_uterina
  - Reseccion_macroscopica_complet
  - Tec_histerec
  - Tratamiento_RT
  - Tratamiento_sistemico
  - Tratamiento_sistemico_realizad
  - Tributaria_a_Radioterapia
  - Tt_recidiva_qx
  - abordajeqx
  - afectacion_linf
  - afectacion_omen
  - ap_gPelv_loc
  - ap_gPor_afect_tot
  - asa
  - beta_cateninap
  - bqt
  - bt_realPac
  - causa_muerte
  - comp_claviendin_mes
  - comp_intraop_r01
  - comp_intraop_r02
  - comp_intraop_r03
  - comp_intraop_r04
  - comp_intraop_r05
  - comp_intraop_r06
  - comp_intraop_r07
  - compl_precoc_r01
  - compl_precoc_r02
  - compl_precoc_r03
  - compl_precoc_r04
  - compl_precoc_r05
  - compl_precoc_r06
  - compl_precoc_r07
 


Tipos de datos después de la conversión:
Columnas categóricas: 134
Columnas numéricas: 27


In [15]:
vars_predictoras_candidatas = [
    "imc", "asa", "valor_de_ca125",
    "histo_defin", "grado_histologi", "FIGO2023",
    "tamano_tumoral", "afectacion_linf", "metasta_distan",
    "AP_centinela_pelvico", "AP_ganPelv", "AP_glanPaor",
    "recep_est_porcent", "rece_de_Ppor", "beta_cateninap",
    
    "tipo_histologico", "Grado", "estadiaje_pre_i"
]

# Filter only the candidate predictor variables that exist in the dataframe
vars_predictoras_candidatas = [col for col in vars_predictoras_candidatas if col in df.columns]

print(f"Variables predictoras candidatas: {len(vars_predictoras_candidatas)}")
print("\nVariables disponibles:")
for col in sorted(vars_predictoras_candidatas):
	print(f"  - {col}")

# Update the original df to only keep predictor variables and survival outcomes
df = df[vars_predictoras_candidatas + ['event', 'time_days', 'edad_en_cirugia']].copy()

print(f"\nDataFrame actualizado:")
print(f"Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")

Variables predictoras candidatas: 18

Variables disponibles:
  - AP_centinela_pelvico
  - AP_ganPelv
  - AP_glanPaor
  - FIGO2023
  - Grado
  - afectacion_linf
  - asa
  - beta_cateninap
  - estadiaje_pre_i
  - grado_histologi
  - histo_defin
  - imc
  - metasta_distan
  - rece_de_Ppor
  - recep_est_porcent
  - tamano_tumoral
  - tipo_histologico
  - valor_de_ca125

DataFrame actualizado:
Dimensiones: 147 filas x 21 columnas


In [16]:
# Define categorical columns from the current predictor variables
# (we will recompute this list later too, after conversions)
categorical_predictoras = [
    col for col in vars_predictoras_candidatas
    if col in df.columns and (df[col].dtype.name == "category")
]

print(f"Variables categóricas en el conjunto de predictores: {len(categorical_predictoras)}")

# ============================================================
# Impute missing values in predictor variables using alternative columns
# (post-op <- pre-op)
# IMPORTANT: do the backfill in STRING space to avoid pandas Categorical dtype issues
# Also create flags to mark values filled from pre-op columns
# NOTE: at this point missing values are still NaN (not 'Missing' strings)
# ============================================================

# Ensure target post-op columns are strings before assignment (avoid category mismatch errors)
for col in ["histo_defin", "grado_histologi", "FIGO2023"]:
    if col in df.columns:
        df[col] = df[col].astype("string")

# histo_defin <- tipo_histologico
if "histo_defin" in df.columns and "tipo_histologico" in df.columns:
    # Create flag for "filled from pre"
    df["histo_defin__from_pre"] = 0

    # Missing is NaN at this stage
    missing_histo = df["histo_defin"].isna()
    fill_mask = missing_histo & df["tipo_histologico"].notna()

    # Backfill and flag
    df.loc[fill_mask, "histo_defin"] = df.loc[fill_mask, "tipo_histologico"].astype("string")
    df.loc[fill_mask, "histo_defin__from_pre"] = 1

    print(f"  - histo_defin: {int(fill_mask.sum())} missings imputados con tipo_histologico")

# grado_histologi <- Grado
if "grado_histologi" in df.columns and "Grado" in df.columns:
    df["grado_histologi__from_pre"] = 0

    missing_grado = df["grado_histologi"].isna()
    fill_mask = missing_grado & df["Grado"].notna()

    df.loc[fill_mask, "grado_histologi"] = df.loc[fill_mask, "Grado"].astype("string")
    df.loc[fill_mask, "grado_histologi__from_pre"] = 1

    print(f"  - grado_histologi: {int(fill_mask.sum())} missings imputados con Grado")

# FIGO2023 <- estadiaje_pre_i
if "FIGO2023" in df.columns and "estadiaje_pre_i" in df.columns:
    df["FIGO2023__from_pre"] = 0

    missing_figo = df["FIGO2023"].isna()
    fill_mask = missing_figo & df["estadiaje_pre_i"].notna()

    df.loc[fill_mask, "FIGO2023"] = df.loc[fill_mask, "estadiaje_pre_i"].astype("string")
    df.loc[fill_mask, "FIGO2023__from_pre"] = 1

    print(f"  - FIGO2023: {int(fill_mask.sum())} missings imputados con estadiaje_pre_i")

# Drop the pre-op columns after backfill (safe even if they do not exist)
df = df.drop(columns=["tipo_histologico", "Grado", "estadiaje_pre_i"], errors="ignore")

# ============================================================
# Convert missing values in categorical columns to a new category 'Missing'
# We will:
#   1) Identify categorical-like predictors (category/object/string/bool)
#   2) Convert them to pandas 'category'
#   3) Add 'Missing' category and fill NaNs
# ============================================================

# Recompute categorical predictors robustly (after possible dtype changes)
categorical_predictoras = [
    col for col in vars_predictoras_candidatas
    if col in df.columns and (df[col].dtype.name in ["category", "object", "string", "bool"])
]

# Convert to category + fill Missing
for col in categorical_predictoras:
    # Convert to category first (strings/objects/bools become categories cleanly)
    df[col] = df[col].astype("category")

    # Count NaNs before filling
    missings_nan = int(df[col].isna().sum())

    # Ensure 'Missing' exists as a category
    if "Missing" not in df[col].cat.categories:
        df[col] = df[col].cat.add_categories(["Missing"])

    # Fill NaNs with 'Missing'
    if missings_nan > 0:
        df[col] = df[col].fillna("Missing")
        print(f"  - {col}: {missings_nan} missings convertidos a 'Missing'")

# Verify that there are no missing values in categorical columns
print(f"\nMissing values en columnas categóricas después de la conversión:")
print(int(df[categorical_predictoras].isna().sum().sum()))

Variables categóricas en el conjunto de predictores: 13
  - histo_defin: 1 missings imputados con tipo_histologico
  - grado_histologi: 6 missings imputados con Grado
  - FIGO2023: 8 missings imputados con estadiaje_pre_i
  - asa: 9 missings convertidos a 'Missing'
  - FIGO2023: 1 missings convertidos a 'Missing'
  - afectacion_linf: 7 missings convertidos a 'Missing'
  - metasta_distan: 4 missings convertidos a 'Missing'
  - AP_centinela_pelvico: 19 missings convertidos a 'Missing'
  - AP_ganPelv: 92 missings convertidos a 'Missing'
  - AP_glanPaor: 115 missings convertidos a 'Missing'
  - beta_cateninap: 10 missings convertidos a 'Missing'

Missing values en columnas categóricas después de la conversión:
0


In [17]:
# Show missing values summary for predictor variables
print(f"\nResumen de valores faltantes en variables predictoras:")
missing_summary = df[categorical_predictoras].isna().sum().sort_values(ascending=False)
missing_pct_pred = (missing_summary / len(df)) * 100

for col in missing_summary[missing_summary > 0].index:
	print(f"  {col:30s} {missing_summary[col]:4d} ({missing_pct_pred[col]:5.2f}%)")

if missing_summary.sum() == 0:
	print("  No hay valores faltantes en las variables predictoras candidatas")


Resumen de valores faltantes en variables predictoras:
  No hay valores faltantes en las variables predictoras candidatas


In [18]:
num_check = ["imc","tamano_tumoral","valor_de_ca125","recep_est_porcent","rece_de_Ppor"]
for c in num_check:
    if c in df.columns:
        print(c, df[c].describe(percentiles=[.01,.05,.5,.95,.99]))

imc count    145.000000
mean      31.002276
std        7.807467
min       16.700000
1%        18.212000
5%        20.920000
50%       30.200000
95%       45.780000
99%       50.568000
max       56.100000
Name: imc, dtype: float64
tamano_tumoral count    129.000000
mean       3.835271
std        4.598228
min        0.000000
1%         0.012800
5%         0.232000
50%        3.000000
95%        9.600000
99%       19.000000
max       38.000000
Name: tamano_tumoral, dtype: float64
valor_de_ca125 count     17.000000
mean      74.617647
std      104.521955
min        3.700000
1%         4.596000
5%         8.180000
50%       24.200000
95%      300.560000
99%      356.752000
max      370.800000
Name: valor_de_ca125, dtype: float64
recep_est_porcent count     86.000000
mean      75.651163
std       27.587195
min        0.000000
1%         0.850000
5%        10.000000
50%       90.000000
95%      100.000000
99%      100.000000
max      100.000000
Name: recep_est_porcent, dtype: float64
rece_de_

In [19]:
import numpy as np

# 1. Apply log1p transformation to tamano_tumoral (handles 0 values)
df['tamano_tumoral'] = np.log1p(df['tamano_tumoral'])
print(f"tamano_tumoral transformado a escala log1p")

# 2. Convert percentage variables to 0-1 scale and clip
for col in ['recep_est_porcent', 'rece_de_Ppor']:
	df[col] = (df[col] / 100).clip(0, 1)
	print(f"{col} convertido a escala 0-1")

# 3. Apply log1p transformation to valor_de_ca125 and cap at 99th percentile
p99_log = np.log1p(df['valor_de_ca125']).quantile(0.99)
df['valor_de_ca125'] = np.log1p(df['valor_de_ca125']).clip(upper=p99_log)
print(f"valor_de_ca125 transformado a log1p y capeado en p99={p99_log:.2f}")

# Show summary statistics of transformed variables
print("\nEstadísticas de variables transformadas:")
transformed_vars = ['tamano_tumoral', 'recep_est_porcent', 'rece_de_Ppor', 'valor_de_ca125']
for var in transformed_vars:
	if var in df.columns:
		print(f"\n{var}:")
		print(df[var].describe())

tamano_tumoral transformado a escala log1p
recep_est_porcent convertido a escala 0-1
rece_de_Ppor convertido a escala 0-1
valor_de_ca125 transformado a log1p y capeado en p99=5.88

Estadísticas de variables transformadas:

tamano_tumoral:


count    129.000000
mean       1.338514
std        0.644597
min        0.000000
25%        1.029619
50%        1.386294
75%        1.609438
max        3.663562
Name: tamano_tumoral, dtype: float64

recep_est_porcent:
count    86.000000
mean      0.756512
std       0.275872
min       0.000000
25%       0.625000
50%       0.900000
75%       0.900000
max       1.000000
Name: recep_est_porcent, dtype: float64

rece_de_Ppor:
count    84.000000
mean      0.684286
std       0.311422
min       0.000000
25%       0.482500
50%       0.800000
75%       0.900000
max       1.000000
Name: rece_de_Ppor, dtype: float64

valor_de_ca125:
count    17.000000
mean      3.598414
std       1.208522
min       1.547563
25%       2.721295
50%       3.226844
75%       4.441474
max       5.875255
Name: valor_de_ca125, dtype: float64


In [20]:
# Create binary missingness indicators for numeric variables with missing values
numeric_with_missing = [col for col in num_check if col in df.columns and df[col].isna().any()]

print(f"Creando indicadores de missingness para {len(numeric_with_missing)} variables numéricas:")
for col in numeric_with_missing:
	miss_col_name = f"{col}_miss"
	df[miss_col_name] = df[col].isna().astype("int8")
	n_miss = df[miss_col_name].sum()
	print(f"  - {miss_col_name}: {n_miss} casos con missing")

# Impute missing values with median for numeric variables
print(f"\nImputando valores faltantes con la mediana:")
for col in numeric_with_missing:
	median_value = df[col].median()
	df[col] = df[col].fillna(median_value)
	print(f"  - {col}: imputado con mediana = {median_value:.2f}")

# Verify no missing values remain in numeric columns
print(f"\nVerificación - Missing values en columnas numéricas:")
missing_numeric = df[num_check].isna().sum().sum()
print(f"Total: {missing_numeric}")

if missing_numeric == 0:
	print("✓ Todas las columnas numéricas han sido imputadas correctamente")

Creando indicadores de missingness para 5 variables numéricas:
  - imc_miss: 2 casos con missing
  - tamano_tumoral_miss: 18 casos con missing
  - valor_de_ca125_miss: 130 casos con missing
  - recep_est_porcent_miss: 61 casos con missing
  - rece_de_Ppor_miss: 63 casos con missing

Imputando valores faltantes con la mediana:
  - imc: imputado con mediana = 30.20
  - tamano_tumoral: imputado con mediana = 1.39
  - valor_de_ca125: imputado con mediana = 3.23
  - recep_est_porcent: imputado con mediana = 0.90
  - rece_de_Ppor: imputado con mediana = 0.80

Verificación - Missing values en columnas numéricas:
Total: 0
✓ Todas las columnas numéricas han sido imputadas correctamente


In [21]:
# Show missing values summary for the entire final dataset (all variables)
print("Resumen de valores faltantes en el dataset final:")
print(f"Dimensiones del dataset: {df.shape[0]} filas x {df.shape[1]} columnas\n")

# Calculate missing values for all columns
missing_all = df.isna().sum().sort_values(ascending=False)
missing_pct_all = (missing_all / len(df)) * 100

# Filter columns with at least one missing value
missing_all_nonzero = missing_all[missing_all > 0]

if len(missing_all_nonzero) > 0:
	print(f"Columnas con valores faltantes: {len(missing_all_nonzero)}\n")
	for col in missing_all_nonzero.index:
		print(f"  {col:30s} {missing_all[col]:4d} ({missing_pct_all[col]:5.2f}%)")
	
	print(f"\nTotal de valores faltantes: {missing_all.sum()}")
	print(f"Porcentaje total de missings: {(missing_all.sum() / (len(df) * len(df.columns))) * 100:.2f}%")
else:
	print("✓ No hay valores faltantes en el dataset final")

Resumen de valores faltantes en el dataset final:
Dimensiones del dataset: 147 filas x 26 columnas

✓ No hay valores faltantes en el dataset final


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 147 entries, 0 to 162
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   imc                        147 non-null    float64 
 1   asa                        147 non-null    category
 2   valor_de_ca125             147 non-null    float64 
 3   histo_defin                147 non-null    category
 4   grado_histologi            147 non-null    category
 5   FIGO2023                   147 non-null    category
 6   tamano_tumoral             147 non-null    float64 
 7   afectacion_linf            147 non-null    category
 8   metasta_distan             147 non-null    category
 9   AP_centinela_pelvico       147 non-null    category
 10  AP_ganPelv                 147 non-null    category
 11  AP_glanPaor                147 non-null    category
 12  recep_est_porcent          147 non-null    float64 
 13  rece_de_Ppor               147 non-null 

In [23]:
# Save the cleaned dataframe to CSV
df.to_csv('Dades/dt_model.csv', index=False)

print(f"DataFrame guardado en 'dt_model.csv'")
print(f"Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")

DataFrame guardado en 'dt_model.csv'
Dimensiones: 147 filas x 26 columnas
